In [1]:
import geopandas as gpd
import pandas as pd
from pandas import DataFrame,Series
from  geopandas import GeoDataFrame
from shapely.geometry import Point
import os
data_path = 'output'
field_to_count='id' # Field that contains a value (not null) in each row.
percentage = 'percentage (%)'
count = 'count'
import warnings
warnings.filterwarnings(action='ignore')
project_crs = 'epsg:3857'

In [29]:
path = "wrm_demographics_2022-12-05.csv"
ir_fields =['description','geometry',]
data_0 = pd.read_csv(path)
data = data_0.drop(ir_fields,axis=1)
# date to day and hour
date = pd.to_datetime((data['date'].str.replace('Z','').str.replace('T',' ')))
# date in pacific time
act_time = 'pacific'
data[act_time] = pd.DatetimeIndex(date).tz_localize('GMT').tz_convert(tz='US/Pacific')
hour,day,day_val  = 'hour','day','day_val'
data[hour],data[day], data[day_val] = data['pacific'].dt.hour, data['pacific'].dt.day_name(),data['pacific'].dt.dayofweek
data

,id,date,date_reported,type,birth_year,gender,race,disability,disability_type,mobility_aid,...,lat,gender_class,age,age_class,mobility_aid_type_class,disability_type_class,pacific,hour,day,day_val
0,4,2021-05-27T22:34:00Z,2021-05-28T22:50:36Z,incident,1971,female,white,no,NaN,no response,...,48.459019,Women,50,45-54,No response,None,2021-05-27 15:34:00-07:00,15,Thursday,3
1,7,2021-05-29T19:46:42Z,2021-05-29T20:01:09Z,incident,2000,no-response,no-response,no,NaN,no response,...,48.433674,No Response,21,18-24,No response,None,2021-05-29 12:46:42-07:00,12,Saturday,5
2,8,2021-05-31T17:54:58Z,2021-05-31T18:00:16Z,amenity,1982,male,white,no,NaN,no response,...,49.288649,Men,39,35-44,No response,None,2021-05-31 10:54:58-07:00,10,Monday,0
3,16,2021-06-04T17:31:46Z,2021-06-04T17:56:33Z,incident,1982,male,white,no,NaN,no response,...,49.281230,Men,39,35-44,No response,None,2021-06-04 10:31:46-07:00,10,Friday,4
4,17,2021-06-04T21:35:00Z,2021-06-04T21:41:11Z,amenity,1993,female,white,no,NaN,no response,...,49.263620,Women,28,25-34,No response,None,2021-06-04 14:35:00-07:00,14,Friday,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,1987,2022-11-27T18:00:00Z,2022-12-04T20:03:10Z,hazard-concern,1975,male,black,no,NaN,no response,...,45.497378,Men,47,45-54,No response,None,2022-11-27 10:00:00-08:00,10,Sunday,6
1384,1988,2022-12-04T19:55:30Z,2022-12-04T20:05:26Z,hazard-concern,1975,female,black,no,NaN,no response,...,45.501452,Women,47,45-54,No response,None,2022-12-04 11:55:30-08:00,11,Sunday,6
1385,1989,2022-12-04T19:55:30Z,2022-12-04T20:08:21Z,amenity,1986,male,white,no,NaN,no response,...,45.496891,Men,36,35-44,No response,None,2022-12-04 11:55:30-08:00,11,Sunday,6
1386,1990,2022-12-04T19:55:30Z,2022-12-04T20:11:31Z,amenity,1962,male,latino,no,NaN,no response,...,45.504402,Men,60,55-64,No response,None,2022-12-04 11:55:30-08:00,11,Sunday,6


In [30]:
def bivariate_analysis(data_to_group:(GeoDataFrame, DataFrame), main_vars:list, secondary_vars:list, field_to_store:str):
    r"""
    It goes over the dictionary and calculate the cross relation between the variable inside
    :param is_staty: to use data from other data saved on the disk
    :param data_to_group: data to work on
    :param field_to_store: which field to retain after groupby and counting
    :param name_analysis: it is used to create new folder if necessary
    :param main_vars:
    :param secondary_vars:
    :return:
    """
    for main_var in main_vars:
        print(f'\n{main_var}')
        new_tables_data = {}
        data1 = data_to_group[~data_to_group[main_var].isin(['No Response', 'no response'])].dropna(subset= main_var)
        for other in secondary_vars:
            print(f'{other}\t',end="")
            data2 = data1[~data_to_group[other].isin(['No Response', 'no response'])].dropna(subset=other)
            sum_field = data2.groupby(main_var).count()[field_to_store].rename('count')
            # GroupBy two questions (one of which is main_var)
            table_00 = DataFrame(data2.groupby([main_var, other]).count()[field_to_store].rename(''))
            # Get the data in readable structure
            table_0 =table_00.unstack(fill_value=0)
            # Clean columns and index
            table_0.columns = table_0.columns.droplevel()

            # get percentage (questions variables for each group)
            old_columns =table_0.columns
            table_2 = table_0.copy()
            table_2[old_columns +' (%)'] = round(table_0.div(sum_field, axis=0) * 100, 1).astype(str) +'%'
            table_2['sum_field'] = sum_field
            new_tables_data[f'{other}'] =table_2.sort_values(by='sum_field',ascending=False)
        with pd.ExcelWriter(f"{data_path}/{main_var}.xlsx") as writer:
            [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables_data.items()]

In [31]:
dict_main =['hour','day','gender_class','disability','disability_type','mobility_aid','age_class','disability_type_class']
other_data = ['type', 'feature_type', 'feature_subtype']
bivariate_analysis(data_to_group=data, main_vars=dict_main, secondary_vars=other_data, field_to_store='id')


hour
type	feature_type	feature_subtype	
day
type	feature_type	feature_subtype	
gender_class
type	feature_type	feature_subtype	
disability
type	feature_type	feature_subtype	
disability_type
type	feature_type	feature_subtype	
mobility_aid
type	feature_type	feature_subtype	
age_class
type	feature_type	feature_subtype	
disability_type_class
type	feature_type	feature_subtype	

### make spatial data

In [32]:
is_export = False
crs = 'EPSG:4326'
spatial_folder = 'shpfiles'
spatial_data = GeoDataFrame(data, geometry=data.apply(lambda x:Point(x['lon'],x['lat']),axis =1),crs=crs)
if is_export:
    spatial_data.to_file(f'{spatial_folder}/WRM.shp')

In [33]:
def add_city_name(city_name:str):
    r"""
    Upload the city's shp file from the disk and add its name by s_join to the corresponding points.
    :param city_name:
    :return:
    """
    city  = gpd.read_file(f'{spatial_folder}/{city_name}.shp').to_crs(crs)
    s_data = spatial_data.sjoin(city)
    s_data['city'] =city_name
    return s_data

In [34]:
# Upload city boundary polygons
data_with_city_name = pd.concat([add_city_name(name) for name in ['Ottawa','victoria']])
data_with_city_name = data_with_city_name[list(spatial_data.columns) + ['city']].to_crs(project_crs)

# Export to shp file
data_with_city_name[act_time]= data_with_city_name[act_time].astype(str)
data_with_city_name.to_file(f'{spatial_folder}/Ottawa_victoria.shp')
data_with_city_name[data_with_city_name['city']=='victoria'].to_file(f'{spatial_folder}/victoria_pnt.shp')

In [35]:
# Create new colum which present the informaion in col_to_encode as numeric number
col_to_encode= 'type'
data_with_city_name[f'en_{col_to_encode}'] = data_with_city_name[col_to_encode].astype('category').cat.codes
data_with_city_name[data_with_city_name['city']=='victoria'].to_file(f'{spatial_folder}/victoria_pnt.shp')


In [36]:
dict_main =['city']
other_data = ['type', 'feature_type', 'feature_subtype']
bivariate_analysis(data_to_group=data_with_city_name, main_vars=dict_main, secondary_vars=other_data, field_to_store='id')


city
type	feature_type	feature_subtype	

# Test area

In [77]:
main_vars= dict_main
data_to_group=data_with_city_name
field_to_store = 'id'
secondary_vars = other_data
for main_var in main_vars:
    print(f'\n{main_var}')
    new_tables_data = {}
    data1 = data_to_group[~data_to_group[main_var].isin(['No Response', 'no response'])]
    sum_field = data1.groupby(main_var).count()[field_to_store].rename('count')
    for other in secondary_vars:
        print(f'{other}\t', end="")
        data2 = data1[~data_to_group[other].isin(['No Response', 'no response'])]
        # GroupBy two questions (one of which is main_var)
        table_00 = DataFrame(data2.groupby([main_var, other]).count()[field_to_store].rename(''))
        # Get the data in readable structure
        table_0 = table_00.unstack(fill_value=0)
        # Clean columns and index
        table_0.columns = table_0.columns.droplevel()

        # get percentage (questions variables for each group)
        old_columns = table_0.columns
        table_2 = table_0.copy()
        table_2[old_columns + ' (%)'] = round(table_0.div(sum_field, axis=0) * 100, 1)
        table_2['sum_field'] = sum_field
        new_tables_data[f'{other}'] = table_2.sort_values(by='sum_field', ascending=False)
    with pd.ExcelWriter(f"{data_path}/{main_var}.xlsx") as writer:
        [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables_data.items()]


city
type	feature_type	feature_subtype	

In [73]:
table_0

type,amenity,hazard-concern,incident
city,,,
Ottawa,64,112,7
victoria,231,367,112


In [76]:
sum_field


,count
city,
Ottawa,183
victoria,710
